## Sentiment Analysis Using NLTK with Short Movie Reviews - Creating & Using Module

In [1]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np

### Creating a Module

#### There are atleast 2 ways to use the saved models.
1. Load the models from their saved location and use them.
2. Create a module with all the models loaded as aprt of the module and use it.
<br>

In this notebook we will create a module and import the module

#### Save the following code into sentiment_mod_all.py. Copy the *py file and all the saved algorithms either into the working directory or to the packages directory 

```Python 
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

from pathlib import Path
currDir = Path.cwd()

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

documents_f = open("pickled_algos/documents_JVR.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()

word_featuresAll_f = open("pickled_algos/word_featuresAll.pickle", "rb")
word_features = pickle.load(word_featuresAll_f)
word_featuresAll_f.close()

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets_f = open("pickled_algos/featuresetsAll.pickle", "rb")
featuresets = pickle.load(featuresets_f)
featuresets_f.close()

open_file = open("pickled_algos/originalnaivebayesAll.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algos/MNB_classifierAll.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algos/BernoulliNB_classifierAll.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algos/LogisticRegression_classifierAll.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algos/LinearSVC_classifierAll.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algos/SGDC_classifierAll.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()

voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)
```

#### Once the file and the saved algorithms are is saved into the correct location we can import it like anyother Python module

In [2]:
import sentiment_mod_all as s

In [3]:
s.sentiment("This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!")

('pos', 0.6)

In [4]:
s.sentiment("This movie was utter junk. There were absolutely 0 pythons. I don't see what the point was at all. Horrible movie, 0/10")

('neg', 1.0)